In [11]:
from kanren import run, var
from kanren import Relation, facts
father = Relation()
mother = Relation()

facts(father, ("Charles", "William"), ("Charles", "Harry"), ("Mark", "Peter"), ("Mark", "Zara")
      , ("Andrew", "Beatrice"), ("Andrew", "Eugenie"), ("Edward", "Louise"), ("Edward", "James")
      , ("Spencer", "Diana"), ("Spencer", "Charles"), ("Philip", "Anne"), ("Philip", "Andrew")
      , ("Philip", "Edward"), ("George", "Elizabeth"), ("George", "Margaret"))

facts(mother, ("Diana", "William"), ("Diana", "Harry"), ("Anne", "Peter"), ("Anne", "Zara")
      , ("Sarah", "Beatrice"), ("Sarah", "Eugenie"), ("Sophie", "Louise"), ("Sophie", "James")
      , ("Kydd", "Diana"), ("Elizabeth", "Charles"), ("Elizabeth", "Anne"), ("Elizabeth", "Andrew")
      , ("Elizabeth", "Edward"), ("Mum", "Elizabeth"), ("Mum", "Margaret"))

man = ["William", "Harry", "Peter", "James", "Charles", "Mark", "Andrew", "Edward", 
       "Spencer", "Philip", "George"]

woman = ["Zara", "Beatrice", "Eugenie", "Louise", "Diana", "Anne", "Sarah", "Sophie", 
         "Kydd", "Elizabeth", "Margaret", "Mum"]


In [12]:
def GrandChild(grandparent):  # 孙
    # 子女的子女
    someone = var()
    someone_parent = var()
    l1 = list(run(0, someone, father(grandparent, someone_parent), father(someone_parent, someone)) +
              run(0, someone, mother(grandparent, someone_parent), father(someone_parent, someone)) +
              run(0, someone, father(grandparent, someone_parent), mother(someone_parent, someone)) +
              run(0, someone, mother(grandparent, someone_parent), mother(someone_parent, someone)))
    return l1

def Son(parent):  # 儿子
    someone = var()
    l1 = list(run(0, someone, father(parent, someone)) +
              run(0, someone, mother(parent, someone)))
    for i in l1:
        if i in woman:
            l1.remove(i)
    return l1

def Daughter(parent):  # 女儿
    someone = var()
    l1 = list(run(0, someone, father(parent, someone)) +
              run(0, someone, mother(parent, someone)))
    l2 = []
    for i in l1:
        if i in woman:
            l2.append(i)
    return l2

def spouse(one):
    another = var()
    son = Son(one)[0]
    l1 = list(run(0, another, father(another, son)) + run(0, another, mother(another, son)))
    l1.remove(one)
    return l1[0]

def Brother(me):  # 同父母的兄弟，亲兄弟
    # 父母的儿子（除自己外）
    my_parent = var()
    l1 = list(run(0, my_parent, father(my_parent, me)))         # 获得父亲
    l2 = Son(l1[0])    # 求出父母的儿子
    l2 = list(set(l2))
    if me in l2:            # 除掉自己
        l2.remove(me)
    return l2

def Sister(me):  # 同父母的姐妹，亲姐妹，除自己外
    my_parent = var()
    l1 = list(run(0, my_parent, father(my_parent, me)))  # 获得父亲
    l2 = Daughter(l1[0])  # 求出父母的女儿
    l2 = list(set(l2))
    if me in l2:  # 除掉自己
        l2.remove(me)
    return l2

def BrotherInLaw(me):  # 姐夫/妹夫
    someone = var()
    l1 = Sister(me) + Sister(spouse(me))  # 我所有的姐妹和我配偶的姐妹
    l2, l3 = [], []
    for i in l1:
        l2 = l2 + Son(i) + Daughter(i)  # 我姐妹所有的子女
    for i in l2:
        l3 = l3 + list(run(0, someone, father(someone, i)))  # 每个子女们的父亲，即姐夫
    l3 = list(set(l3))
    return l3

def SisterInLaw(me):  # 嫂子/弟妹
    someone = var()
    l1 = Brother(me) + Brother(spouse(me))  # 我所有的兄弟和我配偶的兄弟
    l2, l3 = [], []
    for i in l1:
        l2 = l2 + Son(i) + Daughter(i)  # 我兄弟所有的子女
    for i in l2:
        l3 = l3 + list(run(0, someone, mother(someone, i)))  # 每个子女们的母亲，即嫂子/弟妹
    l3 = list(set(l3))
    return l3

def FirstCousin(me):  # 父母的亲兄弟亲姐妹的孩子
    my_parent = var()
    l1 = Brother(list(run(0, my_parent, father(my_parent, me)))[0]) + \
         Sister(list(run(0, my_parent, father(my_parent, me)))[0])
    l2 = Brother(list(run(0, my_parent, mother(my_parent, me)))[0]) + \
         Sister(list(run(0, my_parent, mother(my_parent, me)))[0])
    l3 = []
    for i in l1:
        l3 = l3 + Son(i) + Daughter(i)
    for j in l2:
        l3 = l3 + Son(j) + Daughter(j)
    l3 = list(set(l3))
    return l3


def Aunt(me):  # 与父母亲同辈的女性亲属，姑表亲
    parent = var()
    l1 = FirstCousin(me)
    l2 = []
    for i in l1:
        l2 = l2 + list(run(0, parent, mother(parent, i)))
    l2 = list(set(l2))
    return l2


def Uncle(me):  # 与父母亲同辈的男性亲属
    parent = var()
    l1 = FirstCousin(me)
    l2 = []
    for i in l1:
        l2 = l2 + list(run(0, parent, father(parent, i)))
    l2 = list(set(l2))
    return l2

def GreatGrandparent(grandson):  # 曾祖父母  
    # 父亲的父亲的父亲  母亲的母亲的母亲
    # 父亲的父亲的母亲  母亲的母亲的父亲
    # 父亲的母亲的父亲  母亲的父亲的母亲
    # 父亲的母亲的母亲  母亲的父亲的父亲
    someone = var()
    someone_son = var()
    someone_son_son = var()
    l1 = list(run(0, someone, father(someone, someone_son), father(someone_son, someone_son_son),
                  father(someone_son_son, grandson)) +
              run(0, someone, father(someone, someone_son), father(someone_son, someone_son_son),
                  mother(someone_son_son, grandson)) +
              run(0, someone, father(someone, someone_son), mother(someone_son, someone_son_son),
                  father(someone_son_son, grandson)) +
              run(0, someone, father(someone, someone_son), mother(someone_son, someone_son_son),
                  mother(someone_son_son, grandson)) +
              run(0, someone, mother(someone, someone_son), mother(someone_son, someone_son_son),
                  mother(someone_son_son, grandson)) +
              run(0, someone, mother(someone, someone_son), mother(someone_son, someone_son_son),
                  father(someone_son_son, grandson)) +
              run(0, someone, mother(someone, someone_son), father(someone_son, someone_son_son),
                  mother(someone_son_son, grandson)) +
              run(0, someone, mother(someone, someone_son), father(someone_son, someone_son_son),
                  father(someone_son_son, grandson)))
    return l1

def Ancestor(me):  # 祖先 比爷爷辈还要高一辈的并非血缘关系的所有长辈 曾祖父母及以上
    parent = var()
    l1 = GreatGrandparent(me)
    l2 = []
    l2 = l2 + l1
    while l1:       # 循环判断是否有上一代
        for i in l1:
            l3 = list(run(0, parent, father(parent, i))) + list(run(0, parent, mother(parent, i)))
            # 祖先的父母是否存在于关系中
            if l3:
                l2 = l2 + l3
            l1.remove(i)
    return l2




In [15]:
print("Elizabeth 的孙辈是：",GrandChild("Elizabeth"))
print("Diana 的姐夫／妹夫是：",BrotherInLaw("Diana"))
print("Zara 的曾祖父母是：",GreatGrandparent("Zara"))
print("Eugenie 的祖先是：",Ancestor("Eugenie"))

Elizabeth 的孙辈是： ['Beatrice', 'Harry', 'Louise', 'Eugenie', 'William', 'James', 'Peter', 'Zara']
Diana 的姐夫／妹夫是： ['Charles']
Zara 的曾祖父母是： ['George', 'Mum']
Eugenie 的祖先是： ['George', 'Mum']
